## Efficientnet + distillation

There are private datasets, and this is the first notebook I will be releelasing via cassava competition

Thanks to my team Teo and Varun for their hardwork! If you found this useful, feel free to upvote! Would help me and my team a lot from the hardwork and time we put in!

Lets get started

**Agenda**
1. Imports
2. Setup
3. dataset
4. Augmentations
5. Training loop

**Note** This is the simple notebook, find distillation notebook here
Distillation Notebook: [Here](https://www.kaggle.com/varungadre0910/cassava-trainin-distillation-final)

# Imports

In [ ]:
!pip install efficientnet_pytorch

I know I know.. We are using fastai currently(because it performed the best out of our pytorch and tensorflow models), will be converting this however solely only pytorch

In [ ]:
from fastai.vision.all import *
from fastai.vision.core import *
from fastai.callback.fp16 import *

from fastai.callback.cutmix import *
from torch.distributions.beta import Beta

from fastai.callback.wandb import *

import pandas as pd
import numpy as np

from efficientnet_pytorch import EfficientNet
import albumentations
import wandb

# Setup

Only training for fold 4 because training them all would exceed 8 hour run time

In [ ]:
class Config:
    testing     = False # must be same as create-folds.ipynb
    image_size  = 512
    batch_size  = 16
    epochs      = 17
    f_epochs    = 1
    train_folds = ['f4']
    arch        = 'efficientnet-b4'
    
cfg = Config()

In [ ]:
def set_seeds():
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seeds()

In [ ]:
path_str = '../input/cassava-leaf-disease-merged'

images_path = Path(path_str + '/train')
csv_path = Path(path_str + '/merged.csv')
folds_path = Path('../input/fold-indexes/folds-merged.csv')

full_df = pd.read_csv(csv_path)
folds_df = pd.read_csv(folds_path)

# drop rows so we get an even number for our folds and remove duplicates
full_df = full_df[~full_df['image_id'].isin(['1562043567.jpg', '3551135685.jpg', '2252529694.jpg', '1000015157.jpg', '1000201771.jpg', '100042118.jpg', '1001723730.jpg'])]

# Create a test dataset

In [ ]:
#Insert wandb keys here
len(full_df)

# Augmentation and train functions

Albumentations, in case you don't know is a fast and efficient augmentation library. Check docs [here](https://albumentations.ai/)
Although I won't be going to much in depth, it's very efficient and easy to implement. Give it a shot!


In [ ]:
class AlbumentationsTransform(RandTransform):
    split_idx,order = None, 2
    
    def __init__(self, train_aug, valid_aug): 
        store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)


def get_train_aug(size): 
    return albumentations.Compose([
            albumentations.RandomResizedCrop(size,size),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)
])

def get_valid_aug(size): 
    return albumentations.Compose([
        albumentations.Resize(size, size),
        albumentations.CenterCrop(size, size, p=1.),
], p=1.)

def get_x(row): return images_path/row['image_id']
def get_y(row): return row['label']

In [ ]:
def train(dls, fold):
    
    model = EfficientNet.from_pretrained(cfg.arch, num_classes=5)

    # define learner
    learn = Learner(
        dls=dls,
        model=model,
        opt_func=ranger,
        metrics=accuracy,
        loss_func=LabelSmoothingCrossEntropy(),
        cbs=[
            WandbCallback(log_preds=False, log_model=False, n_preds=10),
            CutMix(),
        ]
    ).to_fp16()
    
    lr = 0.001
    
    if not cfg.testing:
        lr_min, lr_steep = learn.lr_find(show_plot=False)
        lr = round(lr_min, 5)
        print(f'found lr of({lr_min}): {round(lr_min, 5)}')
    
    
    # start model training
    learn.fine_tune(
        cfg.epochs,
        base_lr=lr,
        freeze_epochs=cfg.f_epochs,
    )
    
    learn.export(Path(f'model-{fold}.pkl'))
    
    return learn

# Training
Train models on different folds of our data

In [ ]:
for fold in cfg.train_folds:
    val_index = folds_df[fold].to_numpy()
    
    print(f'started training on {fold}')
    
    train_block = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_x=get_x,
        get_y=get_y,
        splitter=IndexSplitter(val_index),
        item_tfms= [
            AlbumentationsTransform(
                get_train_aug(size=cfg.image_size),
                get_valid_aug(size=cfg.image_size)
            )
        ],
        batch_tfms=[Normalize.from_stats(*imagenet_stats)]
    )

    dls = train_block.dataloaders(full_df, bs=cfg.batch_size)
    learn = train(dls, fold)

print(f'training on {cfg.train_folds} done')